In [1]:
import pandas as pd
import numpy as np

In [2]:
books = pd.read_csv('books.csv')
ratings = pd.read_csv('ratings.csv')
book_tags = pd.read_csv('book_tags.csv')
tags = pd.read_csv('tags.csv')

In [3]:
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [4]:
books['original_publication_year'] = books['original_publication_year'].fillna(-1).apply(lambda x: int(x) if x != -1 else -1)



In [5]:
ratings_rmv_duplicates = ratings.drop_duplicates()
unwanted_users = ratings_rmv_duplicates.groupby('user_id')['user_id'].count()
unwanted_users = unwanted_users[unwanted_users < 3]
unwanted_ratings = ratings_rmv_duplicates[ratings_rmv_duplicates.user_id.isin(unwanted_users.index)]
new_ratings = ratings_rmv_duplicates.drop(unwanted_ratings.index)

In [6]:
new_ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [7]:
new_ratings['title'] = books.set_index('id').title.loc[new_ratings.book_id].values


In [8]:
new_ratings.shape

(963473, 4)

In [9]:
books.shape

(10000, 23)

In [10]:
cleaned_books=pd.merge(new_ratings,books,on='title')

In [11]:
cleaned_books.head()

,book_id_x,user_id,rating,title,id,book_id_y,best_book_id,work_id,books_count,isbn,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,314,5,"The Hunger Games (The Hunger Games, #1)",1,2767052,2767052,2792775,272,439023483,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,1,439,3,"The Hunger Games (The Hunger Games, #1)",1,2767052,2767052,2792775,272,439023483,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
2,1,588,5,"The Hunger Games (The Hunger Games, #1)",1,2767052,2767052,2792775,272,439023483,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
3,1,1169,4,"The Hunger Games (The Hunger Games, #1)",1,2767052,2767052,2792775,272,439023483,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
4,1,1185,4,"The Hunger Games (The Hunger Games, #1)",1,2767052,2767052,2792775,272,439023483,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


In [12]:
cleaned_books.shape

(971247, 26)

In [13]:
rating_count=(cleaned_books.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'}))

In [14]:
rating_count.shape

(9964, 2)

In [19]:
book_with_totalRatingCount = cleaned_books.merge(rating_count, left_on = 'title', right_on = 'title', how = 'left')     


In [20]:
book_with_totalRatingCount.head()

,book_id_x,user_id,rating,title,id,book_id_y,best_book_id,work_id,books_count,isbn,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,totalRatingCount
0,1,314,5,"The Hunger Games (The Hunger Games, #1)",1,2767052,2767052,2792775,272,439023483,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,100
1,1,439,3,"The Hunger Games (The Hunger Games, #1)",1,2767052,2767052,2792775,272,439023483,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,100
2,1,588,5,"The Hunger Games (The Hunger Games, #1)",1,2767052,2767052,2792775,272,439023483,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,100
3,1,1169,4,"The Hunger Games (The Hunger Games, #1)",1,2767052,2767052,2792775,272,439023483,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,100
4,1,1185,4,"The Hunger Games (The Hunger Games, #1)",1,2767052,2767052,2792775,272,439023483,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,100


In [21]:
columns = ['book_id_x','language_code','ratings_count','ratings_5','id','book_id_y','best_book_id','work_id','books_count','work_ratings_count','work_text_reviews_count','ratings_1','ratings_2','ratings_3','ratings_4','small_image_url']
book_dropped_rated = book_with_totalRatingCount.drop(columns, axis=1)



In [125]:
book_dropped_rated.shape

(971247, 11)

In [28]:
popularity_threshold = 100
rating_popular_book = book_dropped_rated.query('totalRatingCount >= @popularity_threshold')
rating_popular_book.shape

(580250, 11)

In [126]:
rating_popular_book.head()

,user_id,rating,title,isbn,isbn13,authors,original_publication_year,original_title,average_rating,image_url,totalRatingCount
0,314,5,"The Hunger Games (The Hunger Games, #1)",439023483,9.780439e+12,Suzanne Collins,2008,The Hunger Games,4.34,https://images.gr-assets.com/books/1447303603m...,100
1,439,3,"The Hunger Games (The Hunger Games, #1)",439023483,9.780439e+12,Suzanne Collins,2008,The Hunger Games,4.34,https://images.gr-assets.com/books/1447303603m...,100
2,588,5,"The Hunger Games (The Hunger Games, #1)",439023483,9.780439e+12,Suzanne Collins,2008,The Hunger Games,4.34,https://images.gr-assets.com/books/1447303603m...,100
3,1169,4,"The Hunger Games (The Hunger Games, #1)",439023483,9.780439e+12,Suzanne Collins,2008,The Hunger Games,4.34,https://images.gr-assets.com/books/1447303603m...,100
4,1185,4,"The Hunger Games (The Hunger Games, #1)",439023483,9.780439e+12,Suzanne Collins,2008,The Hunger Games,4.34,https://images.gr-assets.com/books/1447303603m...,100


In [127]:
rating_popular_book.to_csv('final.csv')

In [29]:

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

In [61]:
from scipy.sparse import csr_matrix
book_cleaned = rating_popular_book.drop_duplicates(['user_id', 'title'])
book_cleaned_pivot = book_cleaned.pivot(index = 'title', columns = 'user_id', values = 'rating').fillna(0)
book_cleaned_matrix = csr_matrix(book_cleaned_pivot.values)


In [131]:
book_cleaned_pivot.to_csv("pivot.csv")

KeyboardInterrupt: 

In [62]:
book_cleaned_pivot.head()

user_id,1,4,7,9,10,11,14,19,22,23,...,53401,53403,53409,53411,53413,53414,53419,53421,53422,53424
title,,,,,,,,,,,,,,,,,,,,,
"Angels (Walsh Family, #3)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'Tis (Frank McCourt, #2)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1,000 Places to See Before You Die",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100 Selected Poems,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
model_knn.fit(book_cleaned_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [129]:
import pickle
knnPickle = open('knnpickle_file', 'wb')
pickle.dump(model_knn, knnPickle)

In [108]:
books.title.unique

<bound method Series.unique of 0                 The Hunger Games (The Hunger Games, #1)
1       Harry Potter and the Sorcerer's Stone (Harry P...
2                                 Twilight (Twilight, #1)
3                                   To Kill a Mockingbird
4                                        The Great Gatsby
                              ...                        
9995                            Bayou Moon (The Edge, #2)
9996    Means of Ascent (The Years of Lyndon Johnson, #2)
9997                                The Mauritius Command
9998    Cinderella Ate My Daughter: Dispatches from th...
9999                                  The First World War
Name: title, Length: 10000, dtype: object>

In [64]:
book_cleaned_pivot.index

Index([' Angels (Walsh Family, #3)', ''Salem's Lot',
       ''Tis (Frank McCourt, #2)', '1,000 Places to See Before You Die',
       '100 Selected Poems', '10th Anniversary (Women's Murder Club, #10)',
       '11/22/63', '11th Hour (Women's Murder Club, #11)',
       '12th of Never (Women's Murder Club, #12)',
       '13 Little Blue Envelopes (Little Blue Envelope, #1)',
       ...
       'Zen Mind, Beginner's Mind: Informal Talks on Zen Meditation and Practice',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry Into Values',
       'Zero Day (John Puller, #1)', 'Zodiac', 'Zoo', 'Zorro',
       'for colored girls who have considered suicide/when the rainbow is enuf',
       'god is Not Great: How Religion Poisons Everything', 'number9dream',
       'محال'],
      dtype='object', name='title', length=5691)

In [119]:
name="To Kill a Mockingbird"
book_name=get_book_info(name)
book_name

'To Kill a Mockingbird'

In [120]:
def get_book_info(name):
    for book in book_cleaned_pivot.index:
        if name in book:
            return book


In [121]:
book_info=book_cleaned_pivot.query('title == [@book_name]').iloc[:].values.reshape(1,-1)
 

In [122]:
book_info

array([[0., 0., 0., ..., 0., 0., 0.]])

In [123]:
distances, indices = model_knn.kneighbors(book_info, n_neighbors = 6)


In [124]:
for i in range(0, len(distances.flatten())):
    if i != 0:
        print('{0}: {1}, with distance of {2}:'.format(i, book_cleaned_pivot.index[indices.flatten()[i]], distances.flatten()[i]))
        
        
        

1: The Great Gatsby, with distance of 0.2966652339964868:
2: The Catcher in the Rye, with distance of 0.3699905318987523:
3: Harry Potter and the Sorcerer's Stone (Harry Potter, #1), with distance of 0.3892592020883805:
4: Of Mice and Men, with distance of 0.4204446294803902:
5: Pride and Prejudice, with distance of 0.4251608152166305:
